In [ ]:
pip install geopandas

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 16.5MB/s 
     |████████████████████████████████| 14.7MB 242kB/s 


In [ ]:
pip install pyproj

In [ ]:
from pyproj import Geod

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import shapely
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
delhi = gpd.read_file('/content/drive/My Drive/Colab Notebooks/new_delhi_wards.json')

In [ ]:
delhi = delhi.drop(columns=['WARD_NO',	'WARD_NAME', 'DISPLAY_NAME'])

In [ ]:
dt=pd.read_csv('/content/drive/My Drive/Colab Notebooks/new_delhi-wards-2019-2-All-HourlyAggregate.csv')

In [ ]:
dt=dt.drop(columns=['standard_deviation_travel_time', 'geometric_mean_travel_time',	'geometric_standard_deviation_travel_time'])

In [ ]:
delhi['MOVEMENT_ID']=delhi['MOVEMENT_ID'].astype('int')

In [ ]:
m1=pd.merge(delhi, dt, left_on='MOVEMENT_ID', right_on='sourceid')

In [ ]:
m1.head()

,MOVEMENT_ID,geometry,sourceid,dstid,hod,mean_travel_time
0,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,1,23,233.60
1,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,0,201.43
2,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,1,179.93
3,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,2,165.50
4,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,3,158.29


In [ ]:
m1['geometry']

0          POLYGON ((77.13228 28.63154, 77.13644 28.62062...
1          POLYGON ((77.13228 28.63154, 77.13644 28.62062...
2          POLYGON ((77.13228 28.63154, 77.13644 28.62062...
3          POLYGON ((77.13228 28.63154, 77.13644 28.62062...
4          POLYGON ((77.13228 28.63154, 77.13644 28.62062...
                                 ...                        
1455927    POLYGON ((77.08111 28.57601, 77.08554 28.57480...
1455928    POLYGON ((77.08111 28.57601, 77.08554 28.57480...
1455929    POLYGON ((77.08111 28.57601, 77.08554 28.57480...
1455930    POLYGON ((77.08111 28.57601, 77.08554 28.57480...
1455931    POLYGON ((77.08111 28.57601, 77.08554 28.57480...
Name: geometry, Length: 1455932, dtype: geometry

In [ ]:
m1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
m1=m1.rename(columns={'geometry':'S_geometry'})

In [ ]:
m1.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1455932 entries, 0 to 1455931
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype   
---  ------            --------------    -----   
 0   MOVEMENT_ID       1455932 non-null  int64   
 1   S_geometry        1455932 non-null  geometry
 2   sourceid          1455932 non-null  int64   
 3   hod               1455932 non-null  int64   
 4   mean_travel_time  1455932 non-null  float64 
dtypes: float64(1), geometry(1), int64(3)
memory usage: 66.6 MB


In [ ]:
m2=pd.merge(delhi, dt, left_on='MOVEMENT_ID', right_on='dstid')

In [ ]:
m2=m2.rename(columns={'geometry':'D_geometry'})

In [ ]:
m2

,MOVEMENT_ID,D_geometry,sourceid,dstid,hod,mean_travel_time
0,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,1,23,233.60
1,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",2,1,0,182.14
2,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",2,1,1,162.93
3,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",2,1,2,156.47
4,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",2,1,3,164.51
...,...,...,...,...,...,...
1455927,290,"POLYGON ((77.08111 28.57601, 77.08554 28.57480...",256,290,19,1267.43
1455928,290,"POLYGON ((77.08111 28.57601, 77.08554 28.57480...",256,290,20,1285.56
1455929,290,"POLYGON ((77.08111 28.57601, 77.08554 28.57480...",256,290,21,1204.79
1455930,290,"POLYGON ((77.08111 28.57601, 77.08554 28.57480...",256,290,22,1001.08


In [ ]:
m2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1455932 entries, 0 to 1455931
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype   
---  ------            --------------    -----   
 0   MOVEMENT_ID       1455932 non-null  int64   
 1   D_geometry        1455932 non-null  geometry
 2   sourceid          1455932 non-null  int64   
 3   dstid             1455932 non-null  int64   
 4   hod               1455932 non-null  int64   
 5   mean_travel_time  1455932 non-null  float64 
dtypes: float64(1), geometry(1), int64(4)
memory usage: 77.8 MB


In [ ]:
del(delhi)

In [ ]:
del(dt)

In [ ]:
m=pd.merge(m1, m2, left_on=['sourceid','dstid','hod'], right_on=['sourceid','dstid','hod'])

In [ ]:
m.head(20)

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y
0,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,1,23,233.60,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",233.60
1,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,0,201.43,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",201.43
2,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,1,179.93,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",179.93
3,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,2,165.50,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",165.50
4,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,3,158.29,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",158.29
5,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,4,154.89,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",154.89
6,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,5,165.38,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",165.38
7,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,6,161.79,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",161.79
8,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,7,191.77,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",191.77
9,1,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",1,2,8,240.36,2,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",240.36


In [ ]:
m_copy=m.copy()

In [ ]:
m_copy['S_geometry']=m_copy['S_geometry'].centroid
m_copy['D_geometry']=m_copy['D_geometry'].centroid

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [ ]:
m_copy

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y
0,1,POINT (77.13180 28.62141),1,1,23,233.60,1,POINT (77.13180 28.62141),233.60
1,1,POINT (77.13180 28.62141),1,2,0,201.43,2,POINT (77.14289 28.61059),201.43
2,1,POINT (77.13180 28.62141),1,2,1,179.93,2,POINT (77.14289 28.61059),179.93
3,1,POINT (77.13180 28.62141),1,2,2,165.50,2,POINT (77.14289 28.61059),165.50
4,1,POINT (77.13180 28.62141),1,2,3,158.29,2,POINT (77.14289 28.61059),158.29
...,...,...,...,...,...,...,...,...,...
1455927,290,POINT (77.08282 28.57317),290,279,16,2281.00,279,POINT (77.06904 28.63870),2281.00
1455928,290,POINT (77.08282 28.57317),290,279,17,2848.11,279,POINT (77.06904 28.63870),2848.11
1455929,290,POINT (77.08282 28.57317),290,281,15,1937.00,281,POINT (77.07672 28.63299),1937.00
1455930,290,POINT (77.08282 28.57317),290,281,16,2190.38,281,POINT (77.07672 28.63299),2190.38


In [ ]:
m.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1455932 entries, 0 to 1455931
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   MOVEMENT_ID_x       1455932 non-null  int64   
 1   S_geometry          1455932 non-null  geometry
 2   sourceid            1455932 non-null  int64   
 3   dstid               1455932 non-null  int64   
 4   hod                 1455932 non-null  int64   
 5   mean_travel_time_x  1455932 non-null  float64 
 6   MOVEMENT_ID_y       1455932 non-null  int64   
 7   D_geometry          1455932 non-null  geometry
 8   mean_travel_time_y  1455932 non-null  float64 
dtypes: float64(2), geometry(2), int64(5)
memory usage: 111.1 MB


In [ ]:
m.to_csv('merged.csv',index=False)

In [ ]:
m_copy.to_csv('merged1.csv', index=False)

In [ ]:
data=pd.read_csv('/content/merged1.csv')

In [ ]:
data

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29
...,...,...,...,...,...,...,...,...,...
1455927,290,POINT (77.08282193508637 28.57316948198418),290,279,16,2281.00,279,POINT (77.06904180662636 28.63870326950842),2281.00
1455928,290,POINT (77.08282193508637 28.57316948198418),290,279,17,2848.11,279,POINT (77.06904180662636 28.63870326950842),2848.11
1455929,290,POINT (77.08282193508637 28.57316948198418),290,281,15,1937.00,281,POINT (77.07672407631429 28.63298613132645),1937.00
1455930,290,POINT (77.08282193508637 28.57316948198418),290,281,16,2190.38,281,POINT (77.07672407631429 28.63298613132645),2190.38


In [ ]:
new=data['S_geometry'].str.split("(", n=1, expand=True)
data["a"]=new[0]
data['source_longitude']=new[1]

In [ ]:
data

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,a,source_longitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,POINT,77.13179986460109 28.621414737282)
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,POINT,77.13179986460109 28.621414737282)
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93,POINT,77.13179986460109 28.621414737282)
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50,POINT,77.13179986460109 28.621414737282)
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29,POINT,77.13179986460109 28.621414737282)
...,...,...,...,...,...,...,...,...,...,...,...
1455927,290,POINT (77.08282193508637 28.57316948198418),290,279,16,2281.00,279,POINT (77.06904180662636 28.63870326950842),2281.00,POINT,77.08282193508637 28.57316948198418)
1455928,290,POINT (77.08282193508637 28.57316948198418),290,279,17,2848.11,279,POINT (77.06904180662636 28.63870326950842),2848.11,POINT,77.08282193508637 28.57316948198418)
1455929,290,POINT (77.08282193508637 28.57316948198418),290,281,15,1937.00,281,POINT (77.07672407631429 28.63298613132645),1937.00,POINT,77.08282193508637 28.57316948198418)
1455930,290,POINT (77.08282193508637 28.57316948198418),290,281,16,2190.38,281,POINT (77.07672407631429 28.63298613132645),2190.38,POINT,77.08282193508637 28.57316948198418)


In [ ]:
new=data['source_longitude'].str.split(" ", n=1, expand=True)
data["Source_Longitude"]=new[0]
data['source_latitude']=new[1]
data.drop(columns=['source_longitude'], inplace=True)

In [ ]:
data.head()

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,a,Source_Longitude,source_latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,POINT,77.13179986460109,28.621414737282)
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,POINT,77.13179986460109,28.621414737282)
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93,POINT,77.13179986460109,28.621414737282)
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50,POINT,77.13179986460109,28.621414737282)
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29,POINT,77.13179986460109,28.621414737282)


In [ ]:
new=data['source_latitude'].str.split(")", n=1, expand=True)
data["Source_Latitude"]=new[0]
data['source_longitude']=new[1]
data.drop(columns=['source_longitude'], inplace=True)

In [ ]:
data.head()

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,a,Source_Longitude,source_latitude,Source_Latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,POINT,77.13179986460109,28.621414737282),28.621414737282
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,POINT,77.13179986460109,28.621414737282),28.621414737282
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93,POINT,77.13179986460109,28.621414737282),28.621414737282
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50,POINT,77.13179986460109,28.621414737282),28.621414737282
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29,POINT,77.13179986460109,28.621414737282),28.621414737282


In [ ]:
data.drop(['source_latitude'],axis=1,inplace=True)

In [ ]:
data.drop(['a'],axis=1,inplace=True)

In [ ]:
data.head()

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,Source_Longitude,Source_Latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,77.13179986460109,28.621414737282
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,77.13179986460109,28.621414737282
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93,77.13179986460109,28.621414737282
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50,77.13179986460109,28.621414737282
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29,77.13179986460109,28.621414737282


In [ ]:
new=data['D_geometry'].str.split("(", n=1, expand=True)
data["a"]=new[0]
data['dst_longitude']=new[1]

In [ ]:
new=data['dst_longitude'].str.split(" ", n=1, expand=True)
data["dst_Longitude"]=new[0]
data['dst_latitude']=new[1]
data.drop(columns=['dst_longitude'], inplace=True)

In [ ]:
data.head(2)

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,Source_Longitude,Source_Latitude,a,dst_Longitude,dst_latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,77.13179986460109,28.621414737282,POINT,77.13179986460109,28.621414737282)
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,77.13179986460109,28.621414737282,POINT,77.14289455633259,28.61058711806038)


In [ ]:
new=data['dst_latitude'].str.split(")", n=1, expand=True)
data["dst_Latitude"]=new[0]
data['dst_longitude']=new[1]
data.drop(columns=['dst_longitude'], inplace=True)

In [ ]:
data.head(2)

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,Source_Longitude,Source_Latitude,a,dst_Longitude,dst_latitude,dst_Latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,77.13179986460109,28.621414737282,POINT,77.13179986460109,28.621414737282),28.621414737282
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,77.13179986460109,28.621414737282,POINT,77.14289455633259,28.61058711806038),28.61058711806038


In [ ]:
data.drop(['dst_latitude'],axis=1,inplace=True)
data.drop(['a'],axis=1,inplace=True)

In [ ]:
data.head()

,MOVEMENT_ID_x,S_geometry,sourceid,dstid,hod,mean_travel_time_x,MOVEMENT_ID_y,D_geometry,mean_travel_time_y,Source_Longitude,Source_Latitude,dst_Longitude,dst_Latitude
0,1,POINT (77.13179986460109 28.621414737282),1,1,23,233.60,1,POINT (77.13179986460109 28.621414737282),233.60,77.13179986460109,28.621414737282,77.13179986460109,28.621414737282
1,1,POINT (77.13179986460109 28.621414737282),1,2,0,201.43,2,POINT (77.14289455633259 28.61058711806038),201.43,77.13179986460109,28.621414737282,77.14289455633259,28.61058711806038
2,1,POINT (77.13179986460109 28.621414737282),1,2,1,179.93,2,POINT (77.14289455633259 28.61058711806038),179.93,77.13179986460109,28.621414737282,77.14289455633259,28.61058711806038
3,1,POINT (77.13179986460109 28.621414737282),1,2,2,165.50,2,POINT (77.14289455633259 28.61058711806038),165.50,77.13179986460109,28.621414737282,77.14289455633259,28.61058711806038
4,1,POINT (77.13179986460109 28.621414737282),1,2,3,158.29,2,POINT (77.14289455633259 28.61058711806038),158.29,77.13179986460109,28.621414737282,77.14289455633259,28.61058711806038


In [ ]:
data.drop(['MOVEMENT_ID_x'], axis=1, inplace=True)
data.drop(['MOVEMENT_ID_y'], axis=1, inplace=True)

In [ ]:
data.drop(['mean_travel_time_y'], axis=1, inplace=True)

In [ ]:
data=data.rename(columns={'mean_travel_time_x':'mean_travel_time'})

In [ ]:
data.insert(4,'S_geometry',data.pop('S_geometry'))

In [ ]:
data.head(2)

,sourceid,dstid,hod,mean_travel_time,S_geometry,D_geometry,Source_Longitude,Source_Latitude,dst_Longitude,dst_Latitude
0,1,1,23,233.60,POINT (77.13179986460109 28.621414737282),POINT (77.13179986460109 28.621414737282),77.13179986460109,28.621414737282,77.13179986460109,28.621414737282
1,1,2,0,201.43,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.13179986460109,28.621414737282,77.14289455633259,28.61058711806038


In [ ]:
data.to_csv('lat-long-data.csv', index=False)